In [179]:
import pandas as pd
import numpy
import scipy
from pandas import DataFrame, Series
from matplotlib import pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

In [127]:
# Read in data saved from Creating Deliverables (in the middle somewhere)
data = pd.read_csv('data6.csv', encoding = 'utf-8', parse_dates = True)
data['year'] = pd.to_datetime(data['year'])
display(data.head())
data.info()

,id,department_name,program_category,category_description,program_name,program_description,measure_type,deliverable,measure_unit,measure_target,estimate_or_actual,year,measure_target_dtype,estimate_or_actual_dtype,Total output cost,deliverable_id
0,0,Department of Health and Human Services Output...,Acute Health Services,This output provides: a range of timely and hi...,Admitted Services,Acute and sub-acute patient services (elective...,Quantity,Total separations – all hospitals,number (000),1774.0,1814.0,2017-01-01,<class 'float'>,<class 'float'>,1010.4,0
1,20,Department of Health and Human Services Output...,Acute Health Services,This output provides: a range of timely and hi...,Admitted Services,Acute and sub-acute patient services (elective...,Quantity,Total separations – all hospitals,number (000),1695.0,1720.0,2016-01-01,<class 'float'>,<class 'float'>,8816.9,0
2,40,Department of Health and Human Services Output...,Acute Health Services,This output provides: a range of timely and hi...,Admitted Services,Acute and sub-acute patient services (elective...,Quantity,Total separations – all hospitals,number (000),1586.0,1636.0,2015-01-01,<class 'float'>,<class 'float'>,8038.0,0
3,57,Department of Health and Human Services Output...,Acute Health Services,This output provides: a range of timely and hi...,Admitted Services,Acute and sub-acute patient services (elective...,Quantity,Total separations – all hospitals,number (000),1477.0,1560.0,2014-01-01,<class 'float'>,<class 'float'>,7520.1,0
4,74,Department of Health and Human Services Output...,Acute Health Services,This output provides: a range of timely and hi...,Admitted Services,Acute and sub-acute patient services (elective...,Quantity,Total separations – all hospitals,number (000),1423.0,1477.0,2013-01-01,<class 'float'>,<class 'float'>,7107.9,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3590 entries, 0 to 3589
Data columns (total 16 columns):
id                          3590 non-null int64
department_name             3590 non-null object
program_category            3590 non-null object
category_description        2963 non-null object
program_name                3590 non-null object
program_description         3590 non-null object
measure_type                3590 non-null object
deliverable                 3590 non-null object
measure_unit                3590 non-null object
measure_target              3590 non-null float64
estimate_or_actual          3590 non-null float64
year                        3590 non-null datetime64[ns]
measure_target_dtype        3590 non-null object
estimate_or_actual_dtype    3590 non-null object
Total output cost           3590 non-null float64
deliverable_id              3590 non-null int64
dtypes: datetime64[ns](1), float64(3), int64(2), object(10)
memory usage: 448.8+ KB


In [193]:
# Calculate sum of Total output cost per department, per year
depts = data[['department_name', 'year', 'Total output cost']].drop_duplicates().copy()
depts = depts.groupby(['department_name', 'year']).sum().reset_index().rename(columns = {'Total output cost':'Sum of Dept Total output cost'})

# Use these calculations to do the same for the previous year, and hence get two two columns
# Add two new columns 'prev_year' and 'prev_year Sum of Dept Total output cost'
tmp = depts[['year', 'department_name', 'Sum of Dept Total output cost']].copy().rename(
    columns = {'Sum of Dept Total output cost':'prev_year Sum of Dept Total output cost', 'year':'prev_year'})
display(tmp.head())
depts['prev_year'] = depts['year'].apply(lambda x: x.replace(year=x.year - 1))
depts = depts.merge(tmp, left_on = ['department_name', 'prev_year'], right_on = ['department_name', 'prev_year'], how = 'left')
depts.sort_values(by = ['department_name', 'year']).head(10)

,prev_year,department_name,prev_year Sum of Dept Total output cost
0,2009-01-01,"Department of Economic Development, Jobs, Transport and Resources Output Performance Measures",1553.40
1,2010-01-01,"Department of Economic Development, Jobs, Transport and Resources Output Performance Measures",1215.60
2,2011-01-01,"Department of Economic Development, Jobs, Transport and Resources Output Performance Measures",1386.80
3,2012-01-01,"Department of Economic Development, Jobs, Transport and Resources Output Performance Measures",1971.70
4,2013-01-01,"Department of Economic Development, Jobs, Transport and Resources Output Performance Measures",2531.04


,department_name,year,Sum of Dept Total output cost,prev_year,prev_year Sum of Dept Total output cost
0,"Department of Economic Development, Jobs, Transport and Resources Output Performance Measures",2009-01-01,1553.40,2008-01-01,NaN
1,"Department of Economic Development, Jobs, Transport and Resources Output Performance Measures",2010-01-01,1215.60,2009-01-01,1553.40
2,"Department of Economic Development, Jobs, Transport and Resources Output Performance Measures",2011-01-01,1386.80,2010-01-01,1215.60
3,"Department of Economic Development, Jobs, Transport and Resources Output Performance Measures",2012-01-01,1971.70,2011-01-01,1386.80
4,"Department of Economic Development, Jobs, Transport and Resources Output Performance Measures",2013-01-01,2531.04,2012-01-01,1971.70
5,"Department of Economic Development, Jobs, Transport and Resources Output Performance Measures",2014-01-01,2995.30,2013-01-01,2531.04
6,"Department of Economic Development, Jobs, Transport and Resources Output Performance Measures",2015-01-01,7702.00,2014-01-01,2995.30
7,"Department of Economic Development, Jobs, Transport and Resources Output Performance Measures",2016-01-01,7726.20,2015-01-01,7702.00
8,"Department of Economic Development, Jobs, Transport and Resources Output Performance Measures",2017-01-01,8639.80,2016-01-01,7726.20
9,Department of Education and Training Output Performance Measures,2012-01-01,11473.80,2011-01-01,NaN


In [194]:
# Merge back into main table and pluck only relevant fields
df = data.copy()
df = df.merge(depts).sort_values(by = ['department_name', 'year'])[['department_name', 'Sum of Dept Total output cost', 'prev_year Sum of Dept Total output cost',
        'program_name', 'program_description', 'Total output cost', 'deliverable', 'year', 'estimate_or_actual', 'measure_unit', 'measure_type', 'deliverable_id']]

# Save year as string to make it easier for later processing
df['year'] = df['year'].dt.year.astype(str)

df.to_csv('flattended_data_for_ap.csv', encoding = 'utf-8') # All data for all years
data2 = df
df.info()
df.head(1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3590 entries, 2719 to 2788
Data columns (total 12 columns):
department_name                            3590 non-null object
Sum of Dept Total output cost              3590 non-null float64
prev_year Sum of Dept Total output cost    3381 non-null float64
program_name                               3590 non-null object
program_description                        3590 non-null object
Total output cost                          3590 non-null float64
deliverable                                3590 non-null object
year                                       3590 non-null object
estimate_or_actual                         3590 non-null float64
measure_unit                               3590 non-null object
measure_type                               3590 non-null object
deliverable_id                             3590 non-null int64
dtypes: float64(4), int64(1), object(7)
memory usage: 364.6+ KB


,department_name,Sum of Dept Total output cost,prev_year Sum of Dept Total output cost,program_name,program_description,Total output cost,deliverable,year,estimate_or_actual,measure_unit,measure_type,deliverable_id
2719,"Department of Economic Development, Jobs, Transport and Resources Output Performance Measures",1553.4,NaN,"Creative Industries Access, Development and Innovation","This output supports the creative industries to deliver economic, social and cultural benefit through: talent and leadership; the creative and business ecology; innovation and social impact; participation and place making; and international engagement. The performance measures below compare targets and expected or actual results from the delivery of programs and services as part of this output:This output supports the creative industries to deliver economic, social and cultural benefit through: talent and leadership; the creative and business ecology; innovation and social impact; participation and place making; and international engagement. The performance measures below compare targets and expected or actual results from the delivery of programs and services as part of this output:",41.6,Artist residencies in schools,2009,39.0,number,Quantity,372


In [195]:
# Show all departments
data2['department_name'].drop_duplicates()

2719    Department of Economic Development, Jobs, Transport and Resources Output Performance Measures
1381    Department of Education and Training Output Performance Measures                             
3583    Department of Environment, Land, Water and Planning Output Performance Measures              
930     Department of Health and Human Services Output Performance Measures                          
3368    Department of Justice and Regulation Output Performance Measures                             
1644    Department of Premier and Cabinet Output Performance Measures                                
2917    Department of Treasury and Finance Output Performance Measures                               
Name: department_name, dtype: object

In [196]:
# Iterating over sub_dataframes of dataframe 'df'.
# Sub dataframes are all records for each value in column_name.
def sub_dataframe_from_all_column_values(column_name, df):
    unique_column_values = df[column_name].drop_duplicates()
    for value in unique_column_values:
        yield df[df[column_name] == value]

In [197]:
# From all sub dataframes that are created with 'sub_dataframe_from_all_column_values(column_name, df)',
# Create a list filled dictionary whose keys are in 'columns_for_dict' and values in sub dataframe.
# 'columns_for_dict' must be a dictionary of the form { <name for dict> : <sub dataframe column name> }
def create_list_of_dicts_from_sub_dataframe(column_name, df, columns_for_dict):
    lst = []
    for sub_df in sub_dataframe_from_all_column_values(column_name, df):
        dct = create_dict_from_dataframe(sub_df, columns_for_dict)
        lst.append(dct)
    return lst

In [198]:
# Create a dictionary from 'colummns_for_dict' from dataframe df
# 'columns_for_dict' must be a dictionary of the form { <name for dict> : <dataframe column name> }
def create_dict_from_dataframe(df, columns_for_dict):
    dct = {}
    for dict_name, col_name in columns_for_dict.items():
        dct[dict_name] = df[col_name].iloc[0]
    return dct

In [201]:
# Return department in specific json format
def get_department(name, current_year = 2017):
    df = pd.read_csv('flattended_data_for_ap.csv', encoding = 'utf-8')
    
    # Core data needed for this task
    df = df[df['department_name'] == name]
    
    # Create department dictionary and base level values
    tmp = df[df['year'] == current_year]
    columns_for_dict = {'name':'department_name', 'current_budget':'Sum of Dept Total output cost', 'prev_budget':'prev_year Sum of Dept Total output cost'}
    dept = create_dict_from_dataframe(tmp, columns_for_dict)
    
    # Iterate over all programs
    programs = []
    for program in sub_dataframe_from_all_column_values('program_name', df):
        
        # Create program dictionary and base level values
        columns_for_dict = {'name':'program_name', 'description':'program_description'}
        prog = create_dict_from_dataframe(program, columns_for_dict)
        
        # Iterate over all years
        columns_for_dict = {'year':'year', 'budget':'Total output cost'}
        prog['budgets'] = create_list_of_dicts_from_sub_dataframe('year', program, columns_for_dict)
        
        # Iterate over all deliverables
        deliverables = []
        for deliverable in sub_dataframe_from_all_column_values('deliverable', program):
            
            # Create deliverable dictionary and base level values
            columns_for_dict = {'name':'deliverable', 'metric_units':'measure_unit', 'metric_type':'measure_type'}
            deliv = create_dict_from_dataframe(program, columns_for_dict)
            
            # Iterate over all years
            columns_for_dict = {'year':'year', 'metric':'estimate_or_actual'}
            deliv['metrics'] = create_list_of_dicts_from_sub_dataframe('year', program, columns_for_dict)
            
            deliverables.append(deliv)
            
        prog['deliverables'] = deliverables
        
        programs.append(prog)
    
    dept['programs'] = programs
    
    return dept

In [211]:
%%time
get_department('Department of Premier and Cabinet Output Performance Measures')

Wall time: 465 ms


{'current_budget': 510.30000000000007,
 'name': 'Department of Premier and Cabinet Output Performance Measures',
 'prev_budget': 329.99999999999994,
 'programs': [{'budgets': [{'budget': 12.800000000000001, 'year': 2009},
    {'budget': 13.5, 'year': 2010},
    {'budget': 12.6, 'year': 2011},
    {'budget': 17.5, 'year': 2012},
    {'budget': 18.039999999999999, 'year': 2013},
    {'budget': 23.100000000000001, 'year': 2014},
    {'budget': 21.800000000000001, 'year': 2015},
    {'budget': 26.199999999999999, 'year': 2016},
    {'budget': 44.100000000000001, 'year': 2017}],
   'deliverables': [{'metric_type': 'Quantity',
     'metric_units': 'number',
     'metrics': [{'metric': 60.0, 'year': 2009},
      {'metric': 65.0, 'year': 2010},
      {'metric': 60.0, 'year': 2011},
      {'metric': 60.0, 'year': 2012},
      {'metric': 112.0, 'year': 2013},
      {'metric': 75.0, 'year': 2014},
      {'metric': 62.0, 'year': 2015},
      {'metric': 74.0, 'year': 2016},
      {'metric': 60.0, '